In [ ]:
import _pickle as pickle
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import timeit

from itertools import product

In [ ]:
n_samples = np.floor(1 * (2 ** np.arange(6, 18)) ** 2)
n_context = 8

p_high = np.arange(1, 10, 2)
p_high = p_high / (p_high + 1)
p_high  = np.concatenate((p_high, np.array([0.99, 0.999])))

print(n_samples)
print(p_high)

In [ ]:
n_high = np.floor(n_samples[:, None] * p_high[None, :])
n_low = np.floor(n_samples[:, None] * (1 - p_high[None, :]))

In [ ]:
iw_constant = 10
error_iw_high = iw_constant / np.sqrt(n_high)
error_iw_low = iw_constant / np.sqrt(n_low)

In [ ]:
p_relevant_high = (n_context - 1) / n_context + p_high / n_context
p_relevant_low = (n_context - 1) / n_context + (1 - p_high) / n_context

n_relevant_high = np.floor(n_high * p_relevant_high)
n_relevant_low = np.floor(n_low * p_relevant_low)

In [ ]:
p_relevant_low, p_relevant_high

In [ ]:
ic_constant = 0.00001
error_ic_high = (1 - p_relevant_high) + ic_constant / np.sqrt(n_relevant_high)
error_ic_low = (1 - p_relevant_low) + ic_constant / np.clip(np.sqrt(n_relevant_low), a_min=1e-7, a_max=np.inf)

In [ ]:
np.concatenate((error_ic_high.T[..., None], error_iw_high.T[..., None]), axis=-1)

In [ ]:
np.concatenate((error_ic_low.T[..., None], error_iw_low.T[..., None]), axis=-1)

In [ ]:
alpha_high = np.argmin(
    np.concatenate((error_ic_high[..., None], error_iw_high[..., None]), axis=-1),
    axis=-1
).T
alpha_low = np.argmin(
    np.concatenate((error_ic_low[..., None], error_iw_low[..., None]), axis=-1),
    axis=-1
).T

In [ ]:
alpha_high

In [ ]:
alpha_low

In [ ]:
n_high[0]

In [ ]:
p_high

In [ ]:
n_samples.shape

In [ ]:
for p_high_i, curr_p_high in enumerate(p_high):
    plt.plot(
        np.log2(n_samples),
        alpha_high[p_high_i] + p_high_i * 0.01,
        label="{:.3f}".format(curr_p_high),
        marker="x",
        linestyle="--",
        alpha=1.0
    )

plt.title("High freq.")
plt.xlabel("$\\log_2$ num. samples")
plt.ylabel("$\\alpha$")
plt.legend()

In [ ]:
for p_low_i, curr_p_high in enumerate(p_high):
    plt.plot(
        np.log2(n_samples),
        alpha_low[p_low_i] + p_low_i * 0.01,
        label="{:.3f}".format(1 - curr_p_high),
        marker="x",
        linestyle="--",
        alpha=1.0
    )

plt.title("Low freq.")
plt.xlabel("$\\log_2$ num. samples")
plt.ylabel("$\\alpha$")
plt.legend()